# 6. 행렬 분해 기법(Matrix Factorization)

이 챕터에서는 협업 필터링의 모델 기반 접근법인 행렬 분해(Matrix Factorization) 기법을 구현합니다. 사용자-아이템 평점 행렬을 저차원의 잠재 요인(latent factor) 행렬로 분해하여 누락된 평점을 예측하는 방식입니다.

## 6.1 필요한 라이브러리 불러오기

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
from collections import defaultdict
from typing import Dict, List

# 상위 디렉토리 경로를 시스템 경로에 추가하여 utils 모듈을 import할 수 있게 함
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# utils 모듈에서 필요한 클래스와 함수를 import
from utils.models import Dataset, RecommendResult, Metrics
from utils.data_loader import DataLoader

## 6.2 행렬 분해 추천 시스템 구현

In [ ]:
class MFRecommender(BaseRecommender):
    """행렬 분해(Matrix Factorization) 기반 추천 시스템

    사용자-아이템 평점 행렬을 저차원의 잠재 요인(latent factor) 행렬로 분해하여
    누락된 평점을 예측하는 방식입니다. 구현된 알고리즘은 SGD(Stochastic Gradient Descent)를
    사용하여 최적화하는 기본적인 행렬 분해 모델입니다.

    모델 수식:
    예측 평점 r̂_ui = μ + bu + bi + p_u · q_i^T
    여기서:
    - μ: 전체 평균 평점
    - bu: 사용자 u의 편향(bias)
    - bi: 아이템 i의 편향(bias)
    - p_u: 사용자 u의 잠재 요인 벡터
    - q_i: 아이템 i의 잠재 요인 벡터

    알고리즘 로직:
    1. 전처리:
       - 사용자 ID와 영화 ID를 내부 인덱스로 매핑
       - 전체 평균(μ) 계산
       - 사용자 요인 행렬(P), 아이템 요인 행렬(Q) 초기화
       - 사용자 편향(bu), 아이템 편향(bi) 초기화

    2. SGD 학습:
       - 모든 학습 데이터에 대해 n_epochs 횟수만큼 반복
       - 각 평점에 대해:
         a. 현재 모델로 평점 예측
         b. 오차(e) 계산
         c. 경사하강법으로 파라미터 업데이트:
            - 사용자 편향(bu) 업데이트
            - 아이템 편향(bi) 업데이트
            - 사용자 요인 벡터(P[u]) 업데이트
            - 아이템 요인 벡터(Q[i]) 업데이트

    3. 평점 예측 (RMSE 계산용):
       - 테스트 데이터의 각 항목에 대해 학습된 모델로 평점 예측
       - 예측값을 0.5~5.0 범위로 제한

    4. 아이템 추천:
       - 각 사용자에 대해 모든 아이템의 예측 평점 계산
       - 이미 평가한 아이템을 제외하고 예측 평점이 높은 상위 k개 추천

    매개변수:
    - k: 추천 목록 크기 (기본값: 10)
    - n_factors: 잠재 요인의 차원 수 (기본값: 20)
    - learning_rate: 학습률 (기본값: 0.01)
    - n_epochs: 반복 학습 횟수 (기본값: 50)
    - reg: 정규화 계수 (기본값: 0.08)

    특징:
    - 메모리 기반 협업 필터링(UserKNN, ItemKNN)보다 일반적으로 더 높은 정확도
    - 대용량 데이터에서도 효율적으로 학습 가능
    - 사용자와 아이템의 잠재적 특성을 학습하여 데이터 희소성 문제를 완화
    """
    def recommend(self, dataset: Dataset, k=10, **kwargs) -> RecommendResult:
        np.random.seed(0)  # 재현성을 위한 시드 고정
        # 하이퍼파라미터 설정
        n_factors = kwargs.get('n_factors', 20)  # 잠재 요인 차원
        lr = kwargs.get('learning_rate', 0.01)  # 학습률
        n_epochs = kwargs.get('n_epochs', 50)   # 반복 학습 횟수
        reg = kwargs.get('reg', 0.08)           # 정규화 계수

        train, test = dataset.train, dataset.test
        mu = float(train['rating'].mean())  # 전체 평균 평점

        # ID를 내부 인덱스로 매핑
        users = sorted(train.userId.unique())
        items = sorted(train.movieId.unique())
        uid2i = {u:i for i,u in enumerate(users)}  # 사용자 ID → 인덱스
        iid2j = {m:j for j,m in enumerate(items)}  # 영화 ID → 인덱스
        nU, nI = len(users), len(items)  # 사용자 수, 아이템 수

        # 모델 파라미터 초기화
        P = 0.1*np.random.randn(nU, n_factors)  # 사용자 요인 행렬
        Q = 0.1*np.random.randn(nI, n_factors)  # 아이템 요인 행렬
        bu = np.zeros(nU)  # 사용자 편향
        bi = np.zeros(nI)  # 아이템 편향

        # 학습 데이터 인덱스 매핑
        df = train.copy()
        df['ui'] = df['userId'].map(uid2i)  # 사용자 ID → 인덱스
        df['ij'] = df['movieId'].map(iid2j)  # 영화 ID → 인덱스

        # SGD로 모델 학습
        for _ in range(n_epochs):
            for r in df.itertuples(index=False):
                u, j, y = int(r.ui), int(r.ij), float(r.rating)
                # 예측값 계산: μ + bu + bi + p_u·q_i^T
                pred = mu + bu[u] + bi[j] + P[u].dot(Q[j])
                # 오차 계산
                e = y - pred
                # 파라미터 업데이트
                bu[u] += lr*(e - reg*bu[u])  # 사용자 편향 업데이트
                bi[j] += lr*(e - reg*bi[j])   # 아이템 편향 업데이트
                Pu = P[u].copy(); Qj = Q[j].copy()  # 원본 보존
                P[u] += lr*(e*Qj - reg*Pu)   # 사용자 요인 업데이트
                Q[j] += lr*(e*Pu - reg*Qj)   # 아이템 요인 업데이트

        # RMSE 계산을 위한 평점 예측
        pred = pd.Series(index=test.index, dtype=float)
        for idx, row in test.iterrows():
            u, m = row['userId'], row['movieId']  # 사용자 ID와 영화 ID
            # 학습 데이터에 없는 사용자나 아이템은 전체 평균 사용
            if (u not in uid2i) or (m not in iid2j):
                pred.at[idx] = mu
            else:
                # 인덱스 매핑
                ui = uid2i[u]; ij = iid2j[m]
                # 예측값 계산: μ + bu + bi + p_u·q_i^T
                pred.at[idx] = mu + bu[ui] + bi[ij] + P[ui].dot(Q[ij])
        # 예측값 범위 조정 (0.5~5.0)
        pred = pred.clip(lower=0.5, upper=5.0)

        # 추천 목록 생성
        seen = train.groupby('userId')['movieId'].apply(set).to_dict()
        user2items = defaultdict(list)
        for u in test['userId'].unique():
            # 학습 데이터에 없는 사용자는 모든 아이템에 평균 점수 부여
            if u not in uid2i:
                scores = {m: mu for m in items}
            else:
                # 모든 아이템에 대한 예측 점수를 한 번에 계산
                ui = uid2i[u]
                s = mu + bu[ui] + bi + Q @ P[ui]  # 행렬 곱을 통한 벡터화 연산
                scores = {items[j]: float(s[j]) for j in range(nI)}
            # 점수가 높은 순서로 상위 k개 아이템 추천
            user2items[u] = _recommend_sorted(scores, seen.get(u,set()), k)

        return RecommendResult(pred, user2items)

## 6.3 행렬 분해 알고리즘 평가

In [ ]:
# 데이터셋 로드
loader = DataLoader(num_users=100)
dataset = loader.load()

# 메트릭스 계산기
mc = MetricCalculator()
K = 10  # 추천 목록 크기

# MF 평가 (다양한 잠재 요인 차원으로 실험)
print("\n=== Matrix Factorization 평가 ===")
mf_results = []
for n_factors in [10, 20, 50]:
    mf_recommender = MFRecommender()
    mf_result = mf_recommender.recommend(
        dataset,
        k=K, 
        n_factors=n_factors, 
        learning_rate=0.02, 
        n_epochs=30, 
        reg=0.08
    )
    metrics = mc.calc(
        dataset.test['rating'].tolist(),
        mf_result.rating.tolist(),
        dataset.test_user2items,
        mf_result.user2items,
        k=K,
        params={"model": "MF", "n_factors": n_factors}
    )
    mf_results.append(metrics)
    print(f"MF (n_factors={n_factors}):", metrics)

# 최고 성능의 MF 선택
best_mf = max(mf_results, key=lambda x: (x.precision_at_k, -x.rmse))
print("Best MF:", best_mf)

## 6.4 행렬 분해 기법의 장단점

### 장점
1. **높은 예측 정확도**: 메모리 기반 방법보다 일반적으로 더 정확한 예측을 제공
2. **확장성**: 대용량 데이터에서도 효율적으로 학습 및 예측 가능
3. **잠재 요인 학습**: 데이터에서 숨겨진 패턴과 잠재적 특성을 자동으로 학습
4. **희소성 문제 완화**: 적은 평가 데이터로도 합리적인 예측 가능
5. **데이터 압축**: 원래 행렬보다 훨씬 작은 공간에 정보 저장 가능

### 단점
1. **블랙박스 특성**: 잠재 요인의 의미를 직관적으로 해석하기 어려움
2. **하이퍼파라미터 튜닝**: 최적의 성능을 위해 많은 파라미터 튜닝이 필요
3. **추천 설명의 어려움**: 추천 결과의 이유를 사용자에게 설명하기 어려움
4. **새로운 사용자/아이템 처리**: 콜드 스타트 문제가 여전히 존재
5. **학습 시간**: 초기 학습에 상대적으로 오랜 시간이 필요

### 성능 최적화 팁
- **최적의 잠재 요인 차원 찾기**: 그리드 서치를 통해 n_factors 최적화
- **학습률과 정규화 계수 조절**: 과적합과 수렴 속도 사이의 균형 조절
- **초기화 전략 개선**: 랜덤 초기화 대신 SVD 기반 초기화 사용
- **조기 종료(Early Stopping)**: 검증 데이터에서 성능이 더 이상 향상되지 않으면 학습 종료
- **암묵적 피드백(Implicit Feedback) 통합**: 평점 외에도 클릭, 시청 시간 등의 추가 정보 활용

### 확장 모델
- **SVD++**: 암묵적 피드백을 통합하여 MF 모델 확장
- **시간적 요소 고려**: 시간에 따른 사용자 취향 변화 모델링
- **Neural Collaborative Filtering**: 딥러닝을 적용하여 비선형 관계 모델링
- **Factorization Machines**: 추가 특성 정보를 포함한 모델링